In [1]:
# Monte-Carlo playoff odds
# Generate my own playoff odds

# For now, I'm focusing on the mechanics of the simulation, and less so on the inputs (e.g., the projected team quality)
# So I'm using 538's win probabilities for each game, rather than computing my own

# I'm also using 538's results/schedule data, because it is so easy to use

import pandas as pd
import numpy as np

In [2]:
# Read in the 538 dataset, which has a row for each game in the current season (played or unplayed)
gms = pd.read_csv('../data/538/mlb-elo/mlb_elo_latest.csv')
gms

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,rating_prob1,rating_prob2,rating1_post,rating2_post,score1,score2
0,2022-10-05,2022,0,NaN,LAD,COL,1591.773446,1470.180018,0.698066,0.301934,...,NaN,NaN,NaN,NaN,0.720863,0.279137,NaN,NaN,NaN,NaN
1,2022-10-05,2022,0,NaN,SEA,DET,1520.313899,1467.665441,0.608551,0.391449,...,NaN,NaN,NaN,NaN,0.608386,0.391614,NaN,NaN,NaN,NaN
2,2022-10-05,2022,0,NaN,SDP,SFG,1514.624339,1531.272369,0.510579,0.489421,...,NaN,NaN,NaN,NaN,0.565241,0.434759,NaN,NaN,NaN,NaN
3,2022-10-05,2022,0,NaN,NYM,WSN,1530.041565,1438.634452,0.660234,0.339766,...,NaN,NaN,NaN,NaN,0.672951,0.327049,NaN,NaN,NaN,NaN
4,2022-10-05,2022,0,NaN,MIL,ARI,1519.148722,1464.922056,0.610713,0.389287,...,NaN,NaN,NaN,NaN,0.630853,0.369147,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,2022-04-07,2022,0,NaN,ATL,CIN,1555.630840,1501.967218,0.609942,0.390058,...,58.198554,53.297336,18.664382,15.512738,0.620108,0.379892,1552.570297,1501.193092,3.0,6.0
2426,2022-04-07,2022,0,NaN,WSN,NYM,1476.319846,1495.202033,0.507365,0.492635,...,46.506602,48.182760,-10.890192,-33.183129,0.495889,0.504111,1467.302390,1522.210391,1.0,5.0
2427,2022-04-07,2022,0,NaN,STL,PIT,1524.880454,1456.114951,0.630416,0.369584,...,57.273136,46.669517,27.921385,2.182563,0.650312,0.349688,1503.439418,1444.031029,9.0,0.0
2428,2022-04-07,2022,0,NaN,KCR,CLE,1480.923133,1501.256999,0.505276,0.494724,...,50.288294,59.572636,7.862364,30.139987,0.476089,0.523911,1473.144618,1491.474766,3.0,1.0


In [3]:
gms.columns

Index(['date', 'season', 'neutral', 'playoff', 'team1', 'team2', 'elo1_pre',
       'elo2_pre', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post',
       'rating1_pre', 'rating2_pre', 'pitcher1', 'pitcher2', 'pitcher1_rgs',
       'pitcher2_rgs', 'pitcher1_adj', 'pitcher2_adj', 'rating_prob1',
       'rating_prob2', 'rating1_post', 'rating2_post', 'score1', 'score2'],
      dtype='object')

In [4]:
# Split out the games that have been played vs those remaining
played = gms.dropna(subset=['score1']) # games that have a score
remain = gms.loc[gms.index.difference(played.index)] # all other games
played.shape, remain.shape

((1288, 26), (1142, 26))

# Define some functions that will be used in the simulation

In [5]:
def compute_standings(gms_played):
    margins = gms_played['score1']-gms_played['score2']
    winners = pd.Series(np.where(margins>0, gms_played['team1'], gms_played['team2']))
    losers  = pd.Series(np.where(margins<0, gms_played['team1'], gms_played['team2']))
    standings = pd.concat([winners.value_counts().rename('W'), losers.value_counts().rename('L')], axis=1)
    return standings

compute_standings(played)

,W,L
NYY,61,25
LAD,56,29
HOU,56,29
NYM,53,33
ATL,52,35
SDP,49,38
MIL,48,39
MIN,48,40
BOS,47,39
PHI,46,40


In [6]:
#  Create a data frame with the league/division mappings, to use to determine playoff berths
divisions = pd.DataFrame({
'SFG': ['N','NW'],
'LAD': ['N','NW'],
'TBD': ['A','AE'],
'MIL': ['N','NC'],
'HOU': ['A','AW'],
'CHW': ['A','AC'],
'BOS': ['A','AE'],
'NYY': ['A','AE'],
'TOR': ['A','AE'],
'OAK': ['A','AW'],
'SEA': ['A','AW'],
'SDP': ['N','NW'],
'ATL': ['N','NE'],
'CIN': ['N','NC'],
'PHI': ['N','NE'],
'STL': ['N','NC'],
'NYM': ['N','NE'],
'ANA': ['A','AW'],
'CLE': ['A','AC'],
'DET': ['A','AC'],
'CHC': ['N','NC'],
'COL': ['N','NW'],
'KCR': ['A','AC'],
'MIN': ['A','AC'],
'FLA': ['N','NE'],
'WSN': ['N','NE'],
'TEX': ['A','AW'],
'PIT': ['N','NC'],
'BAL': ['A','AE'],
'ARI': ['N','NW']
 }).T

divisions.columns = ['lg', 'div']
divisions

,lg,div
SFG,N,NW
LAD,N,NW
TBD,A,AE
MIL,N,NC
HOU,A,AW
CHW,A,AC
BOS,A,AE
NYY,A,AE
TOR,A,AE
OAK,A,AW


In [7]:

def sim_rem_games(remain):
    # Generate a random number for each game
    randoms = pd.Series(np.random.rand(len(remain)))
    randoms.index = remain.index

    # Figure out the winners and losers
    winners = pd.Series(np.where(randoms<remain['rating_prob1'], remain['team1'], remain['team2']))
    losers = pd.Series(np.where(randoms>remain['rating_prob1'], remain['team1'], remain['team2']))

    # Compute and return the standings
    standings = pd.concat([winners.value_counts().rename('W'), losers.value_counts().rename('L')], axis=1).fillna(0)
    for col in standings.columns: # convert to int
        standings[col] = standings[col].astype(int)
    return standings

sim_rem_games(remain)

,W,L
LAD,49,28
HOU,49,28
CHW,49,29
STL,48,26
TOR,46,29
PHI,45,31
NYY,45,31
NYM,43,33
TBD,42,35
MIL,40,35


In [8]:
cur_standings = compute_standings(played)
rem_standings = sim_rem_games(remain)
full_standings = cur_standings+rem_standings
full_standings

,W,L
ANA,75,87
ARI,69,93
ATL,98,64
BAL,77,85
BOS,88,74
CHC,74,88
CHW,93,69
CIN,65,97
CLE,80,82
COL,72,90


In [9]:
# find playoff teams
def find_playoff_teams(standings):
    standings['wpct'] = standings['W'] / (standings['W'] + standings['L'])

    # Merge in the div/lg data
    standings['div'] = divisions['div']
    standings['lg'] = divisions['lg']

    # Rather than model out all the tie-breakers, I'm assuming that they are all random (not exactly true, but close enough),
    # and so I'm just generating a random number for each team, and we break ties by comparing that random num for each of the tied teams.
    # This is *so* much simpler and faster than modeling all the different scenarios.
    # It might be worth modeling them out with 1-2 days left in the season, but for most of the season, I way prefer using the random num to break ties
    standings['rand'] = np.random.rand(len(standings))

    # Now sort, and break ties using the rand
    sorted = standings.sort_values(by=['wpct', 'rand'], ascending=False)

    div_winners =  list(sorted.groupby(['div']).head(1).index.values)

    wc_contenders = sorted[~sorted.index.isin(div_winners)]
    wc_winners = list(wc_contenders.groupby(['lg']).head(3).index.values)


    return div_winners + wc_winners

     

full_standings, find_playoff_teams(full_standings)

(       W    L      wpct div lg      rand
 ANA   75   87  0.462963  AW  A  0.280211
 ARI   69   93  0.425926  NW  N  0.092603
 ATL   98   64  0.604938  NE  N  0.891958
 BAL   77   85  0.475309  AE  A  0.019188
 BOS   88   74  0.543210  AE  A  0.359827
 CHC   74   88  0.456790  NC  N  0.716705
 CHW   93   69  0.574074  AC  A  0.906161
 CIN   65   97  0.401235  NC  N  0.238517
 CLE   80   82  0.493827  AC  A  0.937705
 COL   72   90  0.444444  NW  N  0.994563
 DET   71   91  0.438272  AC  A  0.208284
 FLA   75   87  0.462963  NE  N  0.546185
 HOU  100   62  0.617284  AW  A  0.917531
 KCR   66   96  0.407407  AC  A  0.732718
 LAD  103   59  0.635802  NW  N  0.196007
 MIL   83   79  0.512346  NC  N  0.947460
 MIN   77   85  0.475309  AC  A  0.208857
 NYM   99   63  0.611111  NE  N  0.028320
 NYY  108   54  0.666667  AE  A  0.924856
 OAK   55  107  0.339506  AW  A  0.362195
 PHI   86   76  0.530864  NE  N  0.973957
 PIT   61  101  0.376543  NC  N  0.756969
 SDP   83   79  0.512346  NW  N  0

In [10]:
%%prun -s cumulative # This runs the code profiler, which creates data I can use to find opportunities for me to speed up the code

[find_playoff_teams(full_standings) for _ in range(1000)]
None # This is to suppress printing the output, which is 1000 lines of the same list of teams

         5455930 function calls (5376930 primitive calls) in 2.568 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.568    2.568 {built-in method builtins.exec}
        1    0.000    0.000    2.568    2.568 <string>:3(<module>)
        1    0.004    0.004    2.568    2.568 <string>:3(<listcomp>)
     1000    0.022    0.000    2.564    0.003 <ipython-input-9-a6d4cc684abe>:2(find_playoff_teams)
     2000    0.004    0.000    0.852    0.000 groupby.py:3487(head)
     1000    0.002    0.000    0.694    0.001 _decorators.py:302(wrapper)
     1000    0.008    0.000    0.692    0.001 frame.py:6269(sort_values)
    12000    0.030    0.000    0.665    0.000 frame.py:3463(__getitem__)
     1000    0.029    0.000    0.489    0.000 sorting.py:285(lexsort_indexer)
     3000    0.005    0.000    0.471    0.000 frame.py:3530(_getitem_bool_array)
40000/24000    0.021    0.000    0.444    0.000 groupby.py:9

In [11]:
def finish_one_season(incoming_standings, remain):
    rem_standings = sim_rem_games(remain)
    full_standings = incoming_standings+rem_standings
    playoff_teams = find_playoff_teams(full_standings)
    return playoff_teams

finish_one_season(cur_standings, remain)

['NYY',
 'LAD',
 'HOU',
 'NYM',
 'MIL',
 'CHW',
 'PHI',
 'ATL',
 'BOS',
 'SDP',
 'TBD',
 'CLE']

In [12]:
#
def sim_n_seasons(incoming_standings, remain, n):
    # The first 6 teams in the list are div winners, the rest are WCs
    return pd.DataFrame([{'sim': i, 'finish': 'div' if t[0] < 6 else 'wc', 'tm': t[1]}  
        for i in range(n) 
        for t in enumerate(finish_one_season(incoming_standings, remain)) 
        ])

sim_results = sim_n_seasons(cur_standings, remain, 10)
sim_results

,sim,finish,tm
0,0,div,NYY
1,0,div,ATL
2,0,div,LAD
3,0,div,HOU
4,0,div,MIL
...,...,...,...
115,9,wc,TOR
116,9,wc,SFG
117,9,wc,SDP
118,9,wc,SEA


In [13]:
# Count the number of div/wc/playoff appearances by team from a set of results
def summarize_sim_results(df_results):
    summary = df_results[['tm', 'finish']].value_counts().unstack().fillna(0)
    for col in summary.columns:
        summary[col] = summary[col].apply(int)
    summary['playoffs'] = summary['div'] + summary['wc']
    return summary

summarize_sim_results(sim_results)

finish,div,wc,playoffs
tm,,,
ATL,4,6,10
BOS,0,8,8
CHW,4,2,6
CLE,1,0,1
HOU,10,0,10
LAD,10,0,10
MIL,10,0,10
MIN,5,1,6
NYM,6,4,10


In [14]:
summarize_sim_results(sim_n_seasons(cur_standings, remain, 10))

finish,div,wc,playoffs
tm,,,
ATL,5,4,9
BAL,0,1,1
BOS,0,6,6
CHW,4,2,6
CLE,0,1,1
HOU,10,0,10
LAD,10,0,10
MIL,7,1,8
MIN,6,3,9


# Now put it all together and simulate a large number of seasons
## (Possibly multiple times, for comparison)


In [22]:
#%%prun  -s cumulative

# We want to simulate the rest of the season a large number of times (e.g., 1000 or 100K or more)
# We also want to run multiple of these sets, to observe the variation across different runs
# So we run trials of num_seasons seasons, num_trials times

NUM_SEASONS = 10*1000
NUM_TRIALS = 5

# Collect just the number of playoff appearances for now, since we'll compare playoff appearance percentagees
def run_trial(incoming_standings, remaining_gms, num_seasons):
    return summarize_sim_results(sim_n_seasons(incoming_standings, remaining_gms, num_seasons))['playoffs']

# Run the sims
totals = pd.concat([run_trial(cur_standings, remain, NUM_SEASONS) for _ in range(NUM_TRIALS)], axis=1)

# Fill in blanks (and convert to ints)
totals = totals.fillna(0)
for col in totals.columns: # convert to int
    totals[col] = totals[col].astype(int)

totals

,playoffs,playoffs,playoffs,playoffs,playoffs
tm,,,,,
ANA,217,210,196,189,183
ARI,2,3,2,2,1
ATL,9781,9768,9792,9783,9794
BAL,205,203,231,209,216
BOS,7024,7174,7103,7106,7179
CHC,3,3,3,3,4
CHW,5311,5250,5227,5298,5301
CLE,1852,1715,1775,1745,1763
DET,12,11,9,18,14


In [23]:
# Average across the trials, to compute the playoff appearance frequency
(totals.apply(np.mean, axis=1)/NUM_SEASONS).sort_values(ascending=False)

tm
HOU    1.00000
NYY    1.00000
LAD    0.99986
NYM    0.98004
ATL    0.97836
SDP    0.85042
MIL    0.80258
MIN    0.75782
BOS    0.71172
PHI    0.68580
TOR    0.66342
TBD    0.58378
CHW    0.52774
SEA    0.50206
STL    0.44562
SFG    0.21284
CLE    0.17700
FLA    0.04368
TEX    0.03398
BAL    0.02128
ANA    0.01990
DET    0.00128
CHC    0.00032
ARI    0.00020
CIN    0.00010
COL    0.00010
PIT    0.00008
KCR    0.00002
dtype: float64

In [24]:
# Which teams have the widest range?  How big is that range?
((totals.apply(max, axis=1) - totals.apply(min, axis=1))/NUM_SEASONS).sort_values(ascending=False)

tm
MIN    0.0177
BOS    0.0155
TBD    0.0143
CLE    0.0137
STL    0.0133
TOR    0.0132
SEA    0.0131
PHI    0.0096
CHW    0.0084
MIL    0.0067
SDP    0.0067
SFG    0.0061
FLA    0.0057
NYM    0.0055
TEX    0.0048
ANA    0.0034
BAL    0.0028
ATL    0.0026
DET    0.0009
LAD    0.0003
PIT    0.0003
ARI    0.0002
CIN    0.0002
COL    0.0002
CHC    0.0001
KCR    0.0001
NYY    0.0000
HOU    0.0000
dtype: float64